In [209]:
import plotly.express as px
import jupyter_dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
import pandas as pd
from dash.dependencies import Input, Output

In [210]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [211]:
#declare app 
app = jupyter_dash.JupyterDash(__name__, external_stylesheets=external_stylesheets)

In [212]:
#create data frames from CSV 
df = pd.read_csv('covid_data/country_vaccinations.csv')
df1 = pd.read_csv('covid_data/country_vaccinations_by_manufacturer.csv') 

In [213]:
#create new data frame with total counts for each vaccine in each country
vax_countries = df1.groupby(['location', 'vaccine'])['total_vaccinations'].sum().reset_index()
vax_countries

,location,vaccine,total_vaccinations
0,Chile,Pfizer/BioNTech,80555807
1,Chile,Sinovac,643278800
2,Czechia,Johnson&Johnson,56899
3,Czechia,Moderna,12979313
4,Czechia,Oxford/AstraZeneca,15364645
5,Czechia,Pfizer/BioNTech,114062982
6,France,Johnson&Johnson,407106
7,France,Moderna,63456652
8,France,Oxford/AstraZeneca,157591330
9,France,Pfizer/BioNTech,724166180


In [214]:
#vaccination map figure
map_fig = px.scatter_geo(df, 
                         locations = 'iso_code', 
                         projection = 'orthographic',
                         color = 'country',
                        opacity = .8, 
                        hover_name = 'country', 
                        hover_data = ['total_vaccinations', 'vaccines'])

In [215]:
#dataframe for total vaccinated bar graph 
total_country_vax = df[["country", "total_vaccinations"]]
total_country_vax = total_country_vax.groupby(['country']).max()

In [216]:
#specifying x & y axis and layout of bar graph 
data = [go.Bar(x=total_country_vax.index, y=total_country_vax.total_vaccinations, name='Total Vaccinations by Country')]
layout = go.Layout(title = 'Total Vaccinations by Country')

#create our figure to be plotted
vax_bar_fig = go.Figure(data = data, layout = layout)
vax_bar_fig.update_yaxes(type="log")

In [217]:
#create new dataframe and groupby location & vaccine
df2 = df1.groupby(['location', 'vaccine']).sum()

In [218]:
#pie chart that represents the vaccine breakdown across several countries
for index2 in df2.index.get_level_values(1): 
    labels = df2.index.get_level_values(1)
    values = df2['total_vaccinations']  #may need to specify the specific row? 
vax_pie_fig = go.Figure(data = [go.Pie(labels = labels, values=values)])
vax_pie_fig.show()

In [219]:

app.layout = html.Div([

    html.H1("COVID-19 Vaccination Progress", style={'text-align': 'center'}),
    
    #div for vaccination bar graph 
    html.Div([
        dcc.Graph(id='vaccination_bar', figure=vax_bar_fig),
    ]),
    
    #div to place two models next to each other 
    html.Div([
            #div for vaccine manufacturer pie chart 
             html.Div([
                html.H4("Vaccination Manufacturers Worldwide", style={'text-align': 'center'}), 
                dcc.Graph(id='manufacturer_pie', figure=vax_pie_fig),
            ], className='six columns'),

             #div for vaccination map graph 
            html.Div([
                html.H4("Total Vaccinations Worldwide", style={'text-align': 'center'}), 
                dcc.Graph(id='vaccination_map', figure=map_fig),
            ], className='six columns'),
    ], className='row'),
    
    #div for the pie chart where user can pick country 
    html.Div([
        
        html.H4("Vaccination Manufacturers by Country", style={'text-align': 'center'}), 
        
        html.Div([
            dcc.Dropdown(id="country_manufacturer", 
                        options=[{"label": 'Chile', "value": 'Chile'}, {"label": 'Czechia', "value": 'Czechia'}, {"label": 'France', "value": 'France'}, 
                                 {"label": 'Germany', "value": 'Germany'}, {"label": 'Iceland', "value": 'Iceland'}, {"label": 'Italy', "value": 'Italy'}, 
                                 {"label": 'Latvia', "value": 'Latvia'}, {"label": 'Lithuania', "value": 'Lithuania'}, {"label": 'Romania', "value": 'Romania'}, 
                                 {"label": 'United States', "value": 'United States'}], 
                        multi=False,
                        value="Chile",
                        style={'width': "40%"}
            ),
        ]),
        
        html.Div(id='pie_container', children=[]),
                    
        dcc.Graph(id='country_manufacturer_pie', figure={}),
    ]),
    
    
    #div with drop down of each country to choose from 
    html.Div([
        
         html.H4("Daily Vaccinations by Country", style={'text-align': 'center'}), 
        
        dcc.Dropdown(id="slct_country",
                 options=[
                     {"label": "Afghanistan", "value": "Afghanistan"},{"label": "Albania", "value": "Albania"},{"label": "Algeria", "value": "Algeria"},{"label": "Andorra", "value": "Andorra"},{"label": 'Angola' , "value": 'Angola'}, {"label": 'Anguilla', "value": 'Anguilla'}, 
                     {"label": 'Antigua and Barbuda', "value": 'Antigua and Barbuda'}, {"label": 'Argentina', "value": 'Argentina'}, 
                     {"label": 'Aruba', "value": 'Aruba'},  {"label": 'Australia', "value":'Australia'}, {"label": 'Austria', "value": 'Austria'},
                     {"label": 'Azerbaijan', "value": 'Azerbaijan'},  {"label": 'Bahamas', "value": 'Bahamas'},  {"label": 'Bahrain', "value": 'Bahrain'},  {"label": 'Bangladesh', "value": 'Bangladesh'}, 
                     {"label": 'Barbados', "value": 'Barbados'},  {"label": 'Belarus', "value":'Belarus'},  {"label": 'Belgium', "value": 'Belgium'},  {"label": 'Belize', "value": 'Belize'}, 
                     {"label": 'Bermuda', "value": 'Bermuda'},  {"label": 'Bhutan', "value": 'Bhutan'},  {"label": 'Bolivia', "value": 'Bolivia'},  {"label": 'Bosnia and Herzegovina', "value": 'Bosnia and Herzegovina'}, 
                     {"label": 'Botswana', "value": 'Botswana'}, {"label": 'Brazil', "value": 'Brazil'}, {"label": 'Brunei', "value": 'Brunei'}, {"label": 'Bulgaria', "value": 'Bulgaria'}, 
                     {"label": 'Cambodia', "value": 'Cambodia'}, {"label": 'Cameroon', "value": 'Cameroon'}, {"label": 'Canada', "value": 'Canada'}, {"label": 'Cape Verde', "value": 'Cape Verde'}, 
                     {"label": 'Cayman Islands', "value": 'Cayman Islands'}, {"label": 'Cayman Islands', "value": 'Cayman Islands'}, {"label": 'Chile', "value": 'Chile'}, {"label": 'China', "value": 'China'}, 
                     {"label": 'Colombia', "value": 'Colombia'}, {"label": 'Costa Rica', "value": 'Costa Rica'}, {"label": "Cote d'Ivoire", "value": "Cote d'Ivoire"}, {"label": 'Croatia', "value": 'Croatia'},
                     {"label": 'Curacao', "value": 'Curacao'}, {"label": 'Cyprus', "value": 'Cyprus'}, {"label": 'Czechia', "value": 'Czechia'}, {"label": 'Democratic Republic of Congo', "value": 'Democratic Republic of Congo'}, 
                     {"label": 'Denmark', "value": 'Denmark'}, {"label": 'Dominica', "value": 'Dominica'}, {"label": 'Dominican Republic', "value": 'Dominican Republic'}, {"label": 'Ecuador', "value": 'Ecuador'},
                     {"label": 'Egypt', "value": 'Egypt'}, {"label": 'El Salvador', "value": 'El Salvador'}, {"label": 'England', "value": 'England'}, {"label": 'Equatorial Guinea', "value": 'Equatorial Guinea'}, 
                     {"label": 'Estonia', "value": 'Estonia'}, {"label": 'Eswatini', "value": 'Eswatini'}, {"label": 'Ethiopia', "value": 'Ethiopia'}, {"label": 'Faeroe Islands', "value": 'Faeroe Islands'}, 
                     {"label": 'Falkland Islands', "value": 'Falkland Islands'}, {"label": 'Fiji', "value": 'Fiji'}, {"label": 'Finland', "value": 'Finland'}, {"label": 'France', "value": 'France'}, 
                     {"label": 'Gabon', "value": 'Gabon'}, {"label": 'Gambia', "value": 'Gambia'}, {"label": 'Georgia', "value": 'Georgia'}, {"label": 'Germany', "value": 'Germany'}, 
                     {"label": 'Ghana', "value": 'Ghana'}, {"label": 'Gibraltar', "value": 'Gibraltar'}, {"label": 'Greece', "value": 'Greece'}, {"label": 'Greenland', "value": 'Greenland'}, 
                     {"label": 'Grenada', "value": 'Grenada'}, {"label": 'Guatemala', "value": 'Guatemala'}, {"label": 'Guernsey', "value":'Guernsey' }, {"label": 'Guinea', "value": 'Guinea'}, 
                     {"label": 'Guyana', "value": 'Guyana'}, {"label": 'Honduras', "value": 'Honduras'}, {"label": 'Hong Kong', "value": 'Hong Kong'}, {"label": 'Hungary', "value": 'Hungary'}, 
                     {"label": 'Iceland', "value": 'Iceland'}, {"label": 'India', "value": 'India'}, {"label": 'Indonesia', "value": 'Indonesia'}, {"label": 'Iran', "value": 'Iran'}, 
                     {"label": 'Iraq', "value": 'Iraq'}, {"label": 'Ireland', "value": 'Ireland'}, {"label": 'Isle of Man', "value": 'Isle of Man'}, {"label": 'Israel', "value": 'Israel'}, 
                     {"label": 'Italy', "value": 'Italy'}, {"label": 'Jamaica', "value": 'Jamaica'}, {"label": 'Japan', "value": 'Japan'}, {"label": 'Jersey', "value": 'Jersey'}, 
                     {"label": 'Jordan', "value": 'Jordan'}, {"label": 'Kazakhstan', "value": 'Kazakhstan'}, {"label": 'Kenya', "value": 'Kenya'}, {"label": 'Kosovo', "value": 'Kosovo'}, 
                     {"label": 'Kuwait', "value": 'Kuwait'}, {"label": 'Kyrgyzstan', "value": 'Kyrgyzstan'}, {"label": 'Laos', "value": 'Laos'}, {"label": 'Latvia', "value": 'Latvia'}, 
                     {"label": 'Lebanon', "value": 'Lebanon'}, {"label": 'Lesotho', "value": 'Lesotho'}, {"label": 'Liechtenstein', "value": 'Liechtenstein'}, {"label": 'Lithuania', "value": 'Lithuania'}, 
                     {"label": 'Luxembourg', "value": 'Luxembourg'}, {"label": 'Macao', "value": 'Macao'}, {"label": 'Malawi', "value": 'Malawi'}, {"label": 'Malaysia', "value": 'Malaysia'}, 
                     {"label": 'Maldives', "value": 'Maldives'}, {"label": 'Mali', "value": 'Mali'}, {"label": 'Malta', "value": 'Malta'}, {"label": 'Mauritania', "value": 'Mauritania'}, 
                     {"label": 'Mauritius', "value": 'Mauritius'}, {"label": 'Mexico', "value": 'Mexico'}, {"label": 'Moldova', "value": 'Moldova'}, {"label": 'Monaco', "value": 'Monaco'}, 
                     {"label": 'Mongolia', "value": 'Mongolia'}, {"label": 'Montenegro', "value": 'Montenegro'}, {"label": 'Montserrat', "value": 'Montserrat'}, {"label": 'Morocco', "value": 'Morocco'}, 
                     {"label": 'Mozambique', "value": 'Mozambique'}, {"label": 'Myanmar', "value": 'Myanmar'}, {"label": 'Namibia', "value": 'Namibia'}, {"label": 'Nauru', "value": 'Nauru'}, 
                     {"label": 'Nepal', "value": 'Nepal'}, {"label":'Netherlands', "value":'Netherlands'}, {"label": 'New Zealand', "value": 'New Zealand'}, {"label": 'Nicaragua', "value": 'Nicaragua'}, 
                     {"label": 'Niger', "value": 'Niger'}, {"label": 'Nigeria', "value": 'Nigeria'}, {"label": 'North Macedonia', "value": 'North Macedonia'}, {"label": 'Northern Cyprus', "value": 'Northern Cyprus'}, 
                     {"label": 'Northern Ireland', "value": 'Northern Ireland'}, {"label": 'Norway', "value": 'Norway'}, {"label": 'Oman', "value": 'Oman'}, {"label": 'Pakistan', "value": 'Pakistan'}, 
                     {"label": 'Palestine', "value": 'Palestine'}, {"label": 'Panama', "value": 'Panama'}, {"label": 'Papua New Guinea', "value": 'Papua New Guinea'}, {"label": 'Paraguay', "value": 'Paraguay'}, 
                     {"label": 'Peru', "value": 'Peru'}, {"label": 'Philippines', "value": 'Philippines'}, {"label": 'Poland', "value": 'Poland'}, {"label": 'Portugal', "value": 'Portugal'}, 
                     {"label": 'Qatar', "value": 'Qatar'}, {"label": 'Romania', "value": 'Romania'}, {"label": 'Russia', "value": 'Russia'}, {"label": 'Rwanda', "value": 'Rwanda'}, 
                     {"label": 'Saint Helena', "value": 'Saint Helena'}, {"label": 'Saint Kitts and Nevis', "value": 'Saint Kitts and Nevis'}, {"label": 'Saint Lucia', "value": 'Saint Lucia'}, {"label": 'Saint Vincent and the Grenadines', "value": 'Saint Vincent and the Grenadines'}, 
                     {"label": 'Samoa', "value": 'Samoa'}, {"label": 'San Marino', "value": 'San Marino'}, {"label": 'Sao Tome and Principe', "value": 'Sao Tome and Principe'}, {"label": 'Saudi Arabia', "value": 'Saudi Arabia'}, 
                     {"label": 'Scotland', "value": 'Scotland'}, {"label": 'Senegal', "value": 'Senegal'}, {"label": 'Serbia', "value": 'Serbia'}, {"label": 'Seychelles', "value": 'Seychelles'}, 
                     {"label": 'Sierra Leone', "value": 'Sierra Leone'}, {"label": 'Singapore', "value": 'Singapore'}, {"label": 'Slovakia', "value": 'Slovakia'}, {"label": 'Slovenia', "value": 'Slovenia'}, 
                     {"label": 'Solomon Islands', "value": 'Solomon Islands'}, {"label": 'South Africa', "value": 'South Africa'}, {"label": 'South Korea', "value": 'South Korea'}, {"label": 'South Sudan', "value": 'South Sudan'}, 
                     {"label": 'Spain', "value": 'Spain'}, {"label": 'Sri Lanka', "value": 'Sri Lanka'}, {"label": 'Sudan', "value": 'Sudan'}, {"label": 'Suriname', "value": 'Suriname'}, 
                     {"label": 'Sweden', "value": 'Sweden'}, {"label": 'Switzerland', "value": 'Switzerland'}, {"label": 'Syria', "value": 'Syria'}, {"label": 'Taiwan', "value": 'Taiwan'}, 
                     {"label": 'Thailand', "value": 'Thailand'}, {"label": 'Togo', "value": 'Togo'}, {"label": 'Tonga', "value": 'Tonga'}, {"label": 'Trinidad and Tobago', "value": 'Trinidad and Tobago'}, 
                     {"label": 'Tunisia', "value": 'Tunisia'}, {"label": 'Turkey', "value": 'Turkey'}, {"label": 'Turks and Caicos Islands', "value": 'Turks and Caicos Islands'}, {"label": 'Uganda', "value": 'Uganda'}, 
                     {"label": 'Ukraine', "value": 'Ukraine'}, {"label": 'United Arab Emirates', "value": 'United Arab Emirates'}, {"label": 'United Kingdom', "value": 'United Kingdom'}, {"label": 'United States', "value": 'United States'}, 
                     {"label": 'Uruguay', "value": 'Uruguay'}, {"label": 'Uzbekistan', "value": 'Uzbekistan'}, {"label": 'Venezuela', "value": 'Venezuela'}, {"label": 'Vietnam', "value": 'Vietnam'}, 
                     {"label": 'Wales', "value": 'Wales'}, {"label": 'Zambia', "value": 'Zambia'}, {"label": 'Zimbabwe', "value": 'Zimbabwe'}], 

                 multi=False,
                 value="Afghanistan",
                 style={'width': "40%"}
                 ),
        
        html.Div(id='output_container', children=[]),
        
        #graph 
        dcc.Graph(id='daily_vax_line', figure={}),
        
    ]),
    
    
])

In [220]:
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='daily_vax_line', component_property='figure')],
    [Input(component_id='slct_country', component_property='value')]
)

def update_graph(option_slctd):

    container = "The country you chose was: {}".format(option_slctd)

    dff = df.copy()
    dff = dff[dff["country"] == option_slctd]
    
    line_fig = px.line(dff, x="date", y="daily_vaccinations", title='Daily Vaccinations in {0}'.format(option_slctd))
    
    return container, line_fig



In [221]:
@app.callback(
    [Output(component_id='pie_container', component_property='children'),
     Output(component_id='country_manufacturer_pie', component_property='figure')],
    [Input(component_id="country_manufacturer", component_property='value')]
)

def update_pie(country_selection): 
    container = "The country you chose was: {}".format(country_selection)
    
    country_vax = vax_countries.copy() 
    country_vax = country_vax[country_vax['location'] == country_selection]
    
    labels = country_vax['vaccine']
    values = country_vax['total_vaccinations'] 
    
    manufacturer_pie_fig = go.Figure(data = [go.Pie(labels = labels, values=values)])
    
    return container, manufacturer_pie_fig
    
    

In [222]:
app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/
